In [2]:
import datetime
import pandas as pd
import numpy as np
from graphviz import Digraph 

In [3]:
#读取数据
raw_data = pd.read_csv('advertising_data.csv')

In [4]:
print('{:*^60}'.format('Data overview:'))
print(raw_data.head(2))
print('{:*^60}'.format('Data dtypes:'))
print(raw_data.dtypes)

***********************Data overview:***********************
        date       source   site     channel media  visit
0  2017/5/15     品牌营销_品牌词  品牌词产品     播放器播放标签    PC  17600
1  2017/5/15  手机_品牌营销_品牌词  品牌词广告  15秒前贴片_app   app  15865
************************Data dtypes:************************
date       object
source     object
site       object
channel    object
media      object
visit      object
dtype: object


In [5]:
raw_data['visit'] = raw_data['visit'].replace('-',0).astype(np.int64)
raw_data['date'] = pd.to_datetime(raw_data['date'],format='%Y/%m/%d')
print('{:*^60}'.format('Data overview:'))
print(raw_data.head(2))
print('{:*^60}'.format('Data dtypes:'))
print(raw_data.dtypes)

***********************Data overview:***********************
        date       source   site     channel media  visit
0 2017-05-15     品牌营销_品牌词  品牌词产品     播放器播放标签    PC  17600
1 2017-05-15  手机_品牌营销_品牌词  品牌词广告  15秒前贴片_app   app  15865
************************Data dtypes:************************
date       datetime64[ns]
source             object
site               object
channel            object
media              object
visit               int64
dtype: object


In [7]:
day_summary = raw_data.ix[:,-1].groupby(raw_data.ix[:, 0]).sum() #按天求和汇总
day_change_value = day_summary.diff(1).rename('change')
day_change_rate = (day_change_value / day_summary).round(3).rename('change_rate')
day_summary_total = pd.concat((day_summary,day_change_value,day_change_rate),axis=1)
print('{:*^60}'.format('Data change summary:'))
print(day_summary_total)

********************Data change summary:********************
             visit   change  change_rate
date                                    
2017-05-15  117260      NaN          NaN
2017-05-16  166124  48864.0        0.294
2017-05-17  157727  -8397.0       -0.053
2017-05-18  155805  -1922.0       -0.012
2017-05-19  115644 -40161.0       -0.347
2017-05-20  120833   5189.0        0.043
2017-05-21  123145   2312.0        0.019
2017-05-22  113624  -9521.0       -0.084
2017-05-23  131248  17624.0        0.134
2017-05-24  149783  18535.0        0.124
2017-05-25  112208 -37575.0       -0.335
2017-05-26   98556 -13652.0       -0.139
2017-05-27  125342  26786.0        0.214
2017-05-28  122626  -2716.0       -0.022
2017-05-29  134067  11441.0        0.085
2017-05-30  137391   3324.0        0.024
2017-05-31  150686  13295.0        0.088
2017-06-01   80334 -70352.0       -0.876
2017-06-02   90468  10134.0        0.112
2017-06-03   79892 -10576.0       -0.132
2017-06-04   91720  11828.0        0.

In [8]:
# 指定日期自动下探分解
the_day = pd.datetime(2017, 6, 7)  # 指定要分析的日期
previous_day = the_day - datetime.timedelta(1)  # 自动获取前1天日期
the_data_tmp = raw_data[raw_data['date'] == the_day].rename(columns={'visit': the_day})  # 获得指定日期数据
previous_data_tmp = raw_data[raw_data['date'] == previous_day].rename(columns={'visit': previous_day})  # 获得前1天日期数据

dimension_list = ['source', 'site', 'channel', 'media']  # 指定要分析的维度：4个层级
split_node_list = ['全站']  # 每层分裂节点名称列表
change_list = list()  # 每层分裂节点对应的总变化量
increase_node_list = []  # 每层最大增长贡献最大的1个维度
decrease_node_list = []  # 每层最小增长贡献最大的1个维度

In [9]:
for dimension in dimension_list:  # 遍历每个维度
    # part1
    the_data_merge = the_data_tmp[[dimension, the_day]]  # 获得指定日期的特定维度和访问量
    previous_data_merge = previous_data_tmp[[dimension, previous_day]]  # 获得指定日期前1天的特定维度和访问量
    the_day_groupby = pd.DataFrame(the_data_merge.ix[:, -1].groupby(the_data_merge.ix[:, 0]).sum())  # 对指定日期特定维度汇总求和
    previous_day_groupby = pd.DataFrame(
        previous_data_merge.ix[:, -1].groupby(previous_data_merge.ix[:, 0]).sum())  # 对指定日期前1天特定维度汇总求和
    # part2
    merge_data = pd.merge(the_day_groupby, previous_day_groupby, how='outer', left_index=True,
                          right_index=True)  # 合并2天的数据
    merge_data = merge_data.fillna(0)  # 将缺失值（没有匹配的值）替换为0
    merge_data['change'] = merge_data[the_day] - merge_data[previous_day]  # 计算环比变化量
    merge_data['change_rate'] = merge_data['change'] / merge_data[previous_day]  # 计算环比变化率
    total_chage = merge_data['change'].sum()  # 获得分裂节点的总变化值
    change_list.append(total_chage)  # 将每个节点的变化值加入列表
    # part3
    merge_data = merge_data.sort_values(by='change')  # 按环比变化量正向排序
    max_increase_node = merge_data.ix[-1].name  # 获得增长变化量最大值节点名称
    max_value, max_rate = merge_data.ix[-1][2:4]  # 获得最大值节点变化量以及变化比例
    increase_node_list.append([max_increase_node, int(max_value), max_rate])  # 将最大值信息追加到列表
    # part4
    min_increase_node = merge_data.ix[0].name  # 获得增长变化量最小值节点名称
    min_value, min_rate = merge_data.ix[0][2:4]  # 获得最小值节点变化量以及变化比例
    decrease_node_list.append([min_increase_node, int(min_value), min_rate])  # 将最小值信息追加到列表
    # part5
    if total_chage >= 0:  # 判断为增长方向
        split_node_list.append(max_increase_node)  # 将分裂节点定义为增长最大值节点
        rules_len = len(split_node_list)  # 通过分裂节点的个数判断所处分裂层级
        if rules_len == 2:  # 第二层source，第一层为全站整体
            the_data_tmp = the_data_tmp[the_data_tmp['source'] == max_increase_node]  # 以source为维度过滤出指定日期符合最大节点条件的数据
            previous_data_tmp = previous_data_tmp[
                previous_data_tmp['source'] == max_increase_node]  # 以source为维度过滤出前1天符合最大节点条件的数据
        elif rules_len == 3:  # 第三层site
            the_data_tmp = the_data_tmp[the_data_tmp['site'] == max_increase_node]  # 以site为维度过滤出指定日期符合最大节点条件的数据
            previous_day_data_tmp = previous_data_tmp[
                previous_data_tmp['site'] == max_increase_node]  # 以site为维度过滤出前1天符合最大节点条件的数据
        elif rules_len == 4:  # 第四层channel
            the_data_tmp = the_data_tmp[the_data_tmp['channel'] == max_increase_node]  # 以channel为维度过滤出指定日期符合最大节点条件的数据
            previous_data_tmp = previous_data_tmp[
                previous_data_tmp['channel'] == max_increase_node]  # 以channel为维度过滤出前1天符合最大节点条件的数据
        elif rules_len == 5:  # 第五层media
            the_data_tmp = the_data_tmp[the_data_tmp['media'] == max_increase_node]  # 以media为维度过滤出指定日期符合最大节点条件的数据
            previous_data_tmp = previous_data_tmp[
                previous_data_tmp['media'] == max_increase_node]  # 以media为维度过滤出前1天符合最大节点条件的数据
    # part6
    else:  # 判断为下降方向
        split_node_list.append(min_increase_node)  # 将分裂节点定义为增长最小值节点
        rules_len = len(split_node_list)  # 通过分裂节点的个数判断所处分裂层级
        if rules_len == 2:  # 第二层source
            the_data_tmp = the_data_tmp[the_data_tmp['source'] == min_increase_node]  # 以source为维度过滤出指定日期符合最小节点条件的数据
            previous_data_tmp = previous_data_tmp[
                previous_data_tmp['source'] == min_increase_node]  # 以source为维度过滤出前1天符合最小节点条件的数据
        elif rules_len == 3:  # 第三层site
            the_data_tmp = the_data_tmp[the_data_tmp['site'] == min_increase_node]  # 以site为维度过滤出指定日期符合最大节点条件的数据
            previous_day_data_tmp = previous_data_tmp[
                previous_data_tmp['site'] == min_increase_node]  # 以site为维度过滤出前1天符合最大节点条件的数据
        elif rules_len == 4:  # 第四层channel
            the_data_tmp = the_data_tmp[the_data_tmp['channel'] == min_increase_node]  # 以channel为维度过滤出指定日期符合最大节点条件的数据
            previous_data_tmp = previous_data_tmp[
                previous_data_tmp['channel'] == min_increase_node]  # 以channel为维度过滤出前1天符合最大节点条件的数据
        elif rules_len == 5:  # 第五层media
            the_data_tmp = the_data_tmp[the_data_tmp['media'] == min_increase_node]  # 以media为维度过滤出指定日期符合最大节点条件的数据
            previous_data_tmp = previous_data_tmp[
                previous_data_tmp['media'] == min_increase_node]  # 以media为维度过滤出前1天符合最大节点条件的数据


/Users/liudawei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
/Users/liudawei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
/Users/liudawei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [10]:
# 画图展示自动下探结果
# patr1
node_style = {'fontname': "SimSun", 'shape': 'box'}  # 定义node节点样式
edge_style = {'fontname': "SimHei", 'fontsize': '11'}  # 定义edge节点样式
top_node_style = '<<table><tr><td bgcolor="black"><font color="white">{0}</font></td></tr><tr><td>环比变化量:{1:d}</td></tr><tr><td>环比变化率:{2:.0%}</td></tr></table>>'  # 定义顶部node节点标签样式
left_node_style = '<<table><tr><td bgcolor="chartreuse"><font color="black">{0}</font></td></tr><tr><td>环比变化量:{1}</td></tr><tr><td>环比变化率:{2:.0%}</td></tr></table>>'  # 定义左侧node节点标签样式
right_node_style = '<<table><tr><td bgcolor="lightblue"><font color="black">{0}</font></td></tr><tr><td>环比变化量:{1}</td></tr><tr><td>环比变化率:{2:.0%}</td></tr></table>>'  # 定义右侧node节点标签样式
dot = Digraph(format='png', node_attr=node_style, edge_attr=edge_style)  # 创建有向图

for i in range(4):  # 循环读取每一层
    # part2
    node_name = split_node_list[i]  # 获得分裂节点名称
    node_left, max_value, max_rate = increase_node_list[i]  # 获得增长最大值名称、变化量和变化率
    node_right, min_value, min_rate = decrease_node_list[i]  # 获得增长最小值名称、变化量和变化率
    node_change = change_list[i]  # 获得分裂节点的总变化量-非分裂节点变化量
    node_label_left = left_node_style.format(node_left, max_value, max_rate)  # 左侧节点显示的信息
    node_label_right = right_node_style.format(node_right, min_value, min_rate)  # 右侧节点显示的信息
    # part3
    if i == 0:  # 如果是顶部节点，则单独增加顶部节点信息
        day_data = day_summary_total[day_summary_total.index == the_day]  # 获得顶部节点的数据
        former_data = day_data.ix[0, 1]  # 获得全站总变化量
        node_lable = top_node_style.format(node_name, int(former_data), day_data.ix[0, 2])  # 分别获取顶部节点名称、变化量和变化率
        dot.node(node_name, label=node_lable)  # 增加顶部节点
    # part4
    contribution_rate_1 = float(max_value) / former_data  # 获得左侧变化量贡献率
    contribution_rate_2 = float(min_value) / former_data  # 获得右侧变化量贡献率
    if node_change >= 0:  # 如果为增长，则左侧为正向
        edge_lablel_left = '正向贡献率:{0:.0%}'.format(contribution_rate_1)  # 左侧边的标签信息
        edge_lablel_right = '反向贡献率:{0:.0%}'.format(contribution_rate_2)  # 右侧边的标签信息
        former_data = max_value  # 获得上一层级变化量最大值
    else:  # 如果为下降，则右侧为正向
        edge_lablel_left = '反向贡献率:{0:.0%}'.format(contribution_rate_1)  # 左侧边的标签信息
        edge_lablel_right = '正向贡献率:{0:.0%}'.format(contribution_rate_2)  # 右侧边的标签信息
        former_data = min_value  # 获得上一层级变化量最大值
    # part5
    dot.node(node_left, label=node_label_left)  # 增加左侧节点
    dot.node(node_right, label=node_label_right)  # 增加右侧节点
    dot.edge(node_name, node_left, label=edge_lablel_left, color='chartreuse')  # 增加左侧边
    dot.edge(node_name, node_right, label=edge_lablel_right, color='lightblue')  # 增加右侧边

dot.view('change summary')  # 展示图形结果

/Users/liudawei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


'change summary.png'